In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [7]:
path = os.path.join(os.getcwd(), 'data/risk_sim.csv')

risks = pd.read_csv(path, sep=';')

In [10]:
from backend.company import Company
from backend.risk import Risk
from backend.supplychain import SupplyChain
from backend.types.risk_type import RiskType

In [11]:
company_list = []

for companies in risks['Companies']:
    for company in companies:
        if company.isalpha() and company not in company_list:
            company_list.append(company)

print(list(company_list))

company_obj_list = []

for company in company_list:
    risks_company = []
    for i, affected_companies in enumerate(risks['Companies'], 0):
        if company in affected_companies:
            row = risks.iloc[i]
            impacts = {'delivery assurance': row['delivery assurance'],
                           'customer requirements': row['customer requirements'],
                           'product data': row['product data'],
                           'green conversion': row['green conversion']}
            new_risk = Risk(row['Risks'], row['Disruption'], row['Frequency'], impacts)
            risks_company.append(impacts)

    company_obj = Company(risks_company)
    company_obj_list.append(company_obj)

supply_chain = SupplyChain(company_obj_list)




['a', 'b', 'c']
